In [1]:
if False:
    ! conda install opencv -y -q
    ! conda install tqdm -y -q
    ! conda install keras -y -q
    ! pip install seansutils seaborn -y -q

In [2]:
## import libaries
from __future__ import absolute_import, division, print_function

import pandas as pd
import numpy as np
import cv2
import os, sys
from tqdm import tqdm

import numpy as np
import keras
#from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from seansUtils.research import StatsCallback, ModelSummary
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (15, 8)

Using TensorFlow backend.


In [3]:
# Class for a dataset
class DataSet(object):
    """Dataset class object."""

    def __init__(self, images, labels):
        """Initialize the class."""
        self._images = images
        self._num_examples = images.shape[0]
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0
        self._epochs_changed = False

    @property
    def images(self):
        return self._images

    @property
    def labels(self):
        return self._labels

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed
    
    @property
    def epochs_changed(self):
        if self._epochs_changed:
            self._epochs_changed = False
            return True
        return False

    def next_batch(self, batch_size):
        """Return the next `batch_size` examples from this data set."""
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            self._epochs_changed = True
            # Shuffle the data
            perm = np.arange(self._num_examples)
            np.random.shuffle(perm)
            self._images = self._images[perm]
            self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch

        return self._images[start:end], self._labels[start:end]

In [4]:
def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (256,256))#use you own size
    return img

In [5]:
def LoadData(path, validatation_data_percentage, test_data_percentage):
    assert (validatation_data_percentage>0 and validatation_data_percentage<=50), "Invalid Validation Percentage(1-50)"
    assert (test_data_percentage>0 and test_data_percentage<=50), "Invalid Test Percentage(1-50)"
    assert (validatation_data_percentage+test_data_percentage<100), "Invalid Percentages(validation+test<100)"

    classes = os.listdir(path)[:5]
    
    global n_classes
    n_classes = len(classes)
    
    '''Loading Data'''
    
    n = 0
    images = []
    labels = []
    images_paths = []
    for label in classes:
        for img_file in os.listdir(path + label)[:50]:
            images_paths.append(path+label+"/"+img_file)
            labels.append(n)
        n+=1
        
    for img_path in tqdm(images_paths):
        img = read_img(path+label+"/"+img_file)
        images.append(img)

    images = np.array(images)
    labels = np.array(labels)
    labels = np.identity(len(classes))[labels]
    
    
    total_data_size = len(labels)
    '''Shuffling Data'''
    perm = np.arange(total_data_size)
    np.random.shuffle(perm)
    images = images[perm]
    labels = labels[perm]
    
    '''Splitting Data'''
    validation_data_size = total_data_size * validatation_data_percentage // 100
    test_data_size = total_data_size * test_data_percentage // 100
    
    test_data_images = images[:test_data_size]
    test_data_labels = labels[:test_data_size]
    
    validation_data_images = images[test_data_size:test_data_size+validation_data_size]
    validation_data_labels = labels[test_data_size:test_data_size+validation_data_size]
    
    train_data_images = images[test_data_size+validation_data_size:]
    train_data_labels = labels[test_data_size+validation_data_size:]
        
    return DataSet(train_data_images, train_data_labels), DataSet(validation_data_images, validation_data_labels), DataSet(test_data_images, test_data_labels)

In [6]:
trainset, validationset, testset = LoadData("../data/faces_images/", 15,15)
print ('\t\tImages\t\t\tLabels')
print ('Training:\t', trainset.images.shape,'\t', trainset.labels.shape)
print ('Validation:\t', validationset.images.shape,'\t', validationset.labels.shape)
print ('Testing:\t', testset.images.shape,'\t', testset.labels.shape)

100%|██████████| 192/192 [00:00<00:00, 251.96it/s]

		Images			Labels
Training:	 (136, 256, 256, 3) 	 (136, 5)
Validation:	 (28, 256, 256, 3) 	 (28, 5)
Testing:	 (28, 256, 256, 3) 	 (28, 5)


# Model and parameters and Transfer learning

In [17]:
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


In [8]:
IMG_SIZE=256
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [9]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(trainset.labels.shape[1], activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [12]:
batch_size = 32 
epochs = 1

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(trainset.images)

In [13]:
history = model.fit_generator(validation_data=(validationset.images, validationset.labels)
    train_datagen.flow(trainset.images, trainset.labels, batch_size=batch_size),
    steps_per_epoch=trainset.images.shape[0] // batch_size,
    epochs=epochs
)

Epoch 1/1
4/4 [==============================] - 681s - loss: 9.5471 - acc: 0.2578     


In [15]:
predictions = model.predict(testset.images)


predictions = np.argmax(predictions, axis=1)

NameError: name 'encoder' is not defined

In [22]:
accuracy = np.mean(np.equal(predictions, np.argmax(testset.labels,1)))
print("Test Accuracy:", accuracy)

Test Accuracy: 0.178571428571
